---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [253]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [318]:
def date_sorter2():
    
    global df
    # Your code here
    # create output dataset
    d = {'index': pd.Series(range(0,df.shape[0])), 'day': [1] * df.shape[0], 'month': [1] * df.shape[0], 'year': [0] * df.shape[0]}
    out = pd.DataFrame(data=d)
    # change all to lowercase
    df = df.str.lower()
    
    # Case 1: year 2009; 2010  #18
    x1 = df[df.str.contains(r'2009|2010')].str.\
        extract(r'(?P<date>(?P<year>2009|2010))')
    x1['index'] = df.index[df.str.contains(r'2009|2010')].values
    x1['day'] = [1] * x1.shape[0]
    x1['month'] = [1] * x1.shape[0]
    out = out.sort_values(['index'])
    x1 = x1.sort_values(['index'])
    out.update(x1)
    
    # Case 2: Feb 2009; Sep 2009; Oct 2010
    x4 = df[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\s](\d{1,4})')].str.\
        extract(r'(?P<date>(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\s](?P<year>\d{1,4}))')
    x4['index'] = df.index[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\s](\d{1,4})')].values
    out = out.sort_values(['index'])
    x4 = x4.sort_values(['index'])
    out.update(x4)

    # Case 3: 6/2008; 12/2009
    x5 = df[df.str.contains(r'\d{1,2}[\/]([1-2][0-9]{3})')].str.\
        extract(r'(?P<date>(?P<month>\d{1,2})[\/](?P<year>[1-2][0-9]{3}))')
    x5['index'] = df.index[df.str.contains(r'\d{1,2}[\/]([1-2][0-9]{3})')].values
    x5.month = x5.month.astype('int')
    x5.month[x5.month>12] = 1
    out = out.sort_values(['index'])
    x5 = x5.sort_values(['index'])
    out.update(x5)
    
    # Case 4: Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; #34
    x2 = df[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.](\s?)(\d{1,2})(.?)(\s?)([12][0-9]{3})')].str.\
        extract(r'(?P<date>(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.]\s?(?P<day>\d{1,2}).?\s?(?P<year>[12][0-9]{3}))')
    x2['index'] = df.index[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.](\s?)(\d{1,2})(.?)(\s?)([12][0-9]{3})')].values
    out = out.sort_values(['index'])
    x2 = x2.sort_values(['index'])
    out.update(x2)
    
    # Case 5: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 #69
    x3 = df[df.str.contains(r'(\d{1,2})\s(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.,]\s?([12][0-9]{3})')].str.\
        extract(r'(?P<date>(?P<day>\d{1,2})\s(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.,]\s?(?P<year>[12][0-9]{3}))')
    x3['index'] = df.index[df.str.contains(r'(\d{1,2})\s(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.,]\s?([12][0-9]{3})')].values
    out = out.sort_values(['index'])
    x3 = x3.sort_values(['index'])
    out.update(x3)
    
    # Case 6: Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 #34
   # x2 = df[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.](\s?)(\d{1,2})(th|st|nd)?(.?)(\s?)([12][0-9]{3})')].str.\
   #     extract(r'(?P<date>(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.]\s?(?P<day>\d{1,2})(th|st|nd)?.?\s?(?P<year>[12][0-9]{3}))')
   # x2['index'] = df.index[df.str.contains(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|january|february|march|april|may|june|july|august|september|october|november|december)[\-\s\.](\s?)(\d{1,2})(.?)(\s?)([12][0-9]{3})')].values
    
    # Case 7: 04/20/2009; 04/20/09; 4/20/09; 4/3/09; also - instead of /
    x6 = df[df.str.contains(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')].str.\
        extract(r'(?P<date>(?P<day>\d{1,2})[/-](?P<month>\d{1,2})[/-](?P<year>\d{2,4}))')
    x6['index'] = df.index[df.str.contains(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}')].values
    x6.month = x6.month.astype('int')
    x6.day = x6.day.astype('int')
    x6["hold"] = x6.day
    x6.loc[x6.month > 12, 'day'] = x6.loc[x6.month > 12, 'month']
    x6.loc[x6.month > 12, 'month'] = x6.loc[x6.month > 12, 'hold']
    del x6["hold"]
    out = out.sort_values(['index'])
    x6 = x6.sort_values(['index'])
    out.update(x6)

    # clean month
    out.loc[(out['month']=='dec') | (out['month']=='december')] = 12
    out.loc[(out['month']=='nov') | (out['month']=='november')] = 11
    out.loc[(out['month']=='oct') | (out['month']=='october')] = 10
    out.loc[(out['month']=='sep') | (out['month']=='september')] = 9
    out.loc[(out['month']=='aug') | (out['month']=='august')] = 8
    out.loc[(out['month']=='jul') | (out['month']=='july')] = 7
    out.loc[(out['month']=='jun') | (out['month']=='june')] = 6
    out.loc[(out['month']=='may') | (out['month']=='may')] = 5
    out.loc[(out['month']=='apr') | (out['month']=='april')] = 4
    out.loc[(out['month']=='mar') | (out['month']=='march')] = 3
    out.loc[(out['month']=='feb') | (out['month']=='february')] = 2
    out.loc[(out['month']=='jan') | (out['month']=='january')] = 1
    
    # clean year
    out.year = out.year.astype('int')
    out.loc[(out['year']<100) & (out['year']>20), 'year'] = out.loc[(out['year']<100) & (out['year']>20), 'year'] + 1900
    out.loc[(out['year']<20), 'year'] = out.loc[(out['year']<20) , 'year'] + 2000
    # sort by date
    
    out.month = out.month.astype('int')
    out.day = out.day.astype('int')
    
    out.sort_values(['year', 'month', 'day'], inplace=True)
    answer = pd.Series(out.index)
    #df1 = pd.Series(list(out.index.labels[0]))

    return answer

In [ ]:
def date_sorter():
    
    # Your code here
    # Full date
    global df
    dates_extracted = df.str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>\d?\d)[/|-](?P<year>\d{4}))')
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d?\d)[/|-](?P<day>([0-2]?[0-9])|([3][01]))[/|-](?P<year>\d{2}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    del dates_extracted[3]
    del dates_extracted[4]
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<day>\d?\d) ?(?P<month>[a-zA-Z]{3,})\.?,? (?P<year>\d{4}))'))
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])
    dates_extracted = dates_extracted.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>[a-zA-Z]{3,})\.?-? ?(?P<day>\d\d?)(th|nd|st)?,?-? ?(?P<year>\d{4}))'))
    del dates_extracted[3]
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Without day
    dates_without_day = df[index_left].str.extractall('(?P<origin>(?P<month>[A-Z][a-z]{2,}),?\.? (?P<year>\d{4}))')
    dates_without_day = dates_without_day.append(df[index_left].str.extractall(r'(?P<origin>(?P<month>\d\d?)/(?P<year>\d{4}))'))
    dates_without_day['day'] = 1
    dates_extracted = dates_extracted.append(dates_without_day)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Only year
    dates_only_year = df[index_left].str.extractall(r'(?P<origin>(?P<year>\d{4}))')
    dates_only_year['day'] = 1
    dates_only_year['month'] = 1
    dates_extracted = dates_extracted.append(dates_only_year)
    index_left = ~df.index.isin([x[0] for x in dates_extracted.index])

    # Year
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: '19' + x if len(x) == 2 else x)
    dates_extracted['year'] = dates_extracted['year'].apply(lambda x: str(x))

    # Month
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: x[1:] if type(x) is str and x.startswith('0') else x)
    month_dict = dict({'September': 9, 'Mar': 3, 'November': 11, 'Jul': 7, 'January': 1, 'December': 12,
                       'Feb': 2, 'May': 5, 'Aug': 8, 'Jun': 6, 'Sep': 9, 'Oct': 10, 'June': 6, 'March': 3,
                       'February': 2, 'Dec': 12, 'Apr': 4, 'Jan': 1, 'Janaury': 1,'August': 8, 'October': 10,
                       'July': 7, 'Since': 1, 'Nov': 11, 'April': 4, 'Decemeber': 12, 'Age': 8})
    dates_extracted.replace({"month": month_dict}, inplace=True)
    dates_extracted['month'] = dates_extracted['month'].apply(lambda x: str(x))

    # Day
    dates_extracted['day'] = dates_extracted['day'].apply(lambda x: str(x))

    # Cleaned date
    dates_extracted['date'] = dates_extracted['month'] + '/' + dates_extracted['day'] + '/' + dates_extracted['year']
    dates_extracted['date'] = pd.to_datetime(dates_extracted['date'])

    dates_extracted.sort_values(by='date', inplace=True)
    df1 = pd.Series(list(dates_extracted.index.labels[0]))
    
    return df1# Your answer here